In [1]:
import pandas as pd
import re
import unicodedata

def enlever_accents(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(text))
        if unicodedata.category(c) != 'Mn'
    )

def nettoyer_texte(text):
    return enlever_accents(text.lower())

def nettoyer_colonnes(cols):
    return [nettoyer_texte(c).replace(' ', '_') for c in cols]

def detecter_sexe(titre_str):
    t = nettoyer_texte(titre_str)
    if 'masculin' in t:
        return 'Masculin'
    elif 'feminin' in t:
        return 'Feminin'
    elif 'ensemble sexes' in t or 'ensemble sexe' in t:
        return 'Ensemble'
    return '.'

def detecter_milieu(titre_str):
    t = nettoyer_texte(titre_str)
    if 'non communal' in t:
        return 'NC'
    elif 'communal' in t:
        return 'C'
    elif 'ensemble milieux' in t or 'ensemble milieu' in t:
        return 'Ensemble'
    return '.'

def detecter_unite(titre_str):
    t = nettoyer_texte(titre_str)
    if '%' in titre_str or 'pourcentage' in t:
        return '%'
    elif 'nombre' in t or 'effectif' in t:
        return 'effectif'
    return 'effectif'

def trouver_colonne_similaire(df, mots_cles):
    for col in df.columns:
        col_nettoyee = nettoyer_texte(col)
        if all(mot in col_nettoyee for mot in mots_cles):
            return col
    return None

fichier_excel = 'educ.xlsx'
xls = pd.ExcelFile(fichier_excel)

pattern = re.compile(r'^T72-(\d{2})$')
feuilles = [f for f in xls.sheet_names if pattern.match(f) and 55 <= int(pattern.match(f).group(1)) <= 89]

dfs_long = []

for feuille in feuilles:
    try:
        print(f"📄 Traitement de la feuille : {feuille}")
        titres = pd.read_excel(xls, sheet_name=feuille, nrows=2, header=None)
        titre_str = ' '.join(titres.astype(str).fillna('').values.flatten())

        sexe = detecter_sexe(titre_str)
        milieu = detecter_milieu(titre_str)
        unite = detecter_unite(titre_str)

        annee_match = re.search(r'\b(19\d{2}|20\d{2})\b', titre_str)
        if annee_match:
            annee = int(annee_match.group(1))
        else:
            match = pattern.match(feuille)
            annee = int("19" + match.group(1)) if match else '.'

        df = pd.read_excel(xls, sheet_name=feuille, skiprows=2)
        df.columns = nettoyer_colonnes(df.columns)

        # Recherche colonnes gouvernorat et groupe d'âge
        gouv_col = trouver_colonne_similaire(df, ['gouver'])
        groupe_age_col = trouver_colonne_similaire(df, ['groupe', 'age'])

        # Si la colonne gouvernorat n'existe pas, on la crée avec '.'
        if gouv_col is None:
            df['gouvernorat'] = '.'
            gouv_col = 'gouvernorat'
        else:
            # Remplace NaN et chaînes vides par '.'
            df[gouv_col] = df[gouv_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        # Idem pour groupe d'âge
        if groupe_age_col is None:
            df['groupe_d_age'] = '.'
            groupe_age_col = 'groupe_d_age'
        else:
            df[groupe_age_col] = df[groupe_age_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        colonnes_instruction = [c for c in df.columns if c not in [gouv_col, 'total', groupe_age_col]]

        df_long = df.melt(
            id_vars=[gouv_col, groupe_age_col],
            value_vars=colonnes_instruction,
            var_name="niveaux_d_etude",
            value_name="valeur"
        )

        # Normaliser noms colonnes
        if gouv_col != 'gouvernorat':
            df_long = df_long.rename(columns={gouv_col: 'gouvernorat'})
            gouv_col = 'gouvernorat'
        if groupe_age_col != 'groupe_d_age':
            df_long = df_long.rename(columns={groupe_age_col: 'groupe_d_age'})
            groupe_age_col = 'groupe_d_age'

        df_long['sexe'] = sexe
        df_long['milieu'] = milieu
        df_long['annee'] = annee
        df_long['unite'] = unite
        df_long['feuille'] = feuille

        dfs_long.append(df_long)
        print(f"✅ Feuille '{feuille}' traitée.\n")

    except Exception as e:
        print(f"❌ Erreur dans la feuille '{feuille}': {e}\n")

if dfs_long:
    df_final = pd.concat(dfs_long, ignore_index=True)
    print("✅ Fusion terminée. Exemple :\n")
    print(df_final.head())
    df_final.to_excel("resultat_educ_auto.xlsx", index=False)
    print("📁 Données enregistrées dans 'resultat_educ_auto.xlsx'")

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df_final)
else:
    print("❌ Aucune donnée traitée.")


📄 Traitement de la feuille : T72-55
✅ Feuille 'T72-55' traitée.

📄 Traitement de la feuille : T72-56
✅ Feuille 'T72-56' traitée.

📄 Traitement de la feuille : T72-57
✅ Feuille 'T72-57' traitée.

📄 Traitement de la feuille : T72-58
✅ Feuille 'T72-58' traitée.

📄 Traitement de la feuille : T72-59
✅ Feuille 'T72-59' traitée.

📄 Traitement de la feuille : T72-60
✅ Feuille 'T72-60' traitée.

📄 Traitement de la feuille : T72-61
✅ Feuille 'T72-61' traitée.

📄 Traitement de la feuille : T72-62
✅ Feuille 'T72-62' traitée.

📄 Traitement de la feuille : T72-63
✅ Feuille 'T72-63' traitée.

📄 Traitement de la feuille : T72-64
✅ Feuille 'T72-64' traitée.

📄 Traitement de la feuille : T72-65
✅ Feuille 'T72-65' traitée.

📄 Traitement de la feuille : T72-66
✅ Feuille 'T72-66' traitée.

📄 Traitement de la feuille : T72-67
✅ Feuille 'T72-67' traitée.

📄 Traitement de la feuille : T72-68
✅ Feuille 'T72-68' traitée.

📄 Traitement de la feuille : T72-69
✅ Feuille 'T72-69' traitée.

📄 Traitement de la feuill

# code version finale



In [1]:
import pandas as pd
import re
import unicodedata

def enlever_accents(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(text))
        if unicodedata.category(c) != 'Mn'
    )

def nettoyer_texte(text):
    return enlever_accents(text.lower())

def nettoyer_colonnes(cols):
    return [nettoyer_texte(c).replace(' ', '_') for c in cols]

def detecter_sexe(titre_str):
    t = nettoyer_texte(titre_str)
    if 'masculin' in t:
        return 'Masculin'
    elif 'feminin' in t:
        return 'Feminin'
    elif 'ensemble sexes' in t or 'ensemble sexe' in t:
        return 'Ensemble'
    return '.'

def detecter_milieu(titre_str):
    t = nettoyer_texte(titre_str)
    if 'non communal' in t:
        return 'NC'
    elif 'communal' in t:
        return 'C'
    elif 'ensemble milieux' in t or 'ensemble milieu' in t:
        return 'Ensemble'
    return '.'

def detecter_unite(titre_str):
    t = nettoyer_texte(titre_str)
    if '%' in titre_str or 'pourcentage' in t:
        return '%'
    elif 'nombre' in t or 'effectif' in t:
        return 'effectif'
    return 'effectif'

def trouver_colonne_similaire(df, mots_cles):
    for col in df.columns:
        col_nettoyee = nettoyer_texte(col)
        if all(mot in col_nettoyee for mot in mots_cles):
            return col
    return None

fichier_excel = 'educ.xlsx'
xls = pd.ExcelFile(fichier_excel)

pattern = re.compile(r'^T72-(\d{2})$')
feuilles = [f for f in xls.sheet_names if pattern.match(f) and 55 <= int(pattern.match(f).group(1)) <= 89]

dfs_long = []

for feuille in feuilles:
    try:
        print(f"📄 Traitement de la feuille : {feuille}")
        titres = pd.read_excel(xls, sheet_name=feuille, nrows=2, header=None)
        titre_str = ' '.join(titres.astype(str).fillna('').values.flatten())

        sexe = detecter_sexe(titre_str)
        milieu = detecter_milieu(titre_str)
        unite = detecter_unite(titre_str)

        annee_match = re.search(r'\b(19\d{2}|20\d{2})\b', titre_str)
        if annee_match:
            annee = int(annee_match.group(1))
        else:
            match = pattern.match(feuille)
            annee = int("19" + match.group(1)) if match else '.'

        df = pd.read_excel(xls, sheet_name=feuille, skiprows=2)
        df.columns = nettoyer_colonnes(df.columns)

        # Recherche colonnes gouvernorat et groupe d'âge
        gouv_col = trouver_colonne_similaire(df, ['gouver'])
        groupe_age_col = trouver_colonne_similaire(df, ['groupe', 'age'])

        # Si la colonne gouvernorat n'existe pas, on la crée avec '.'
        if gouv_col is None:
            df['gouvernorat'] = '.'
            gouv_col = 'gouvernorat'
        else:
            # Remplace NaN et chaînes vides par '.'
            df[gouv_col] = df[gouv_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        # Idem pour groupe d'âge
        if groupe_age_col is None:
            df['groupe_d_age'] = '.'
            groupe_age_col = 'groupe_d_age'
        else:
            df[groupe_age_col] = df[groupe_age_col].fillna('.').astype(str).replace(r'^\s*$', '.', regex=True)

        colonnes_instruction = [c for c in df.columns if c not in [gouv_col, 'total', groupe_age_col]]

        df_long = df.melt(
            id_vars=[gouv_col, groupe_age_col],
            value_vars=colonnes_instruction,
            var_name="niveaux_d_etude",
            value_name="valeur"
        )

        # Normaliser noms colonnes
        if gouv_col != 'gouvernorat':
            df_long = df_long.rename(columns={gouv_col: 'gouvernorat'})
            gouv_col = 'gouvernorat'
        if groupe_age_col != 'groupe_d_age':
            df_long = df_long.rename(columns={groupe_age_col: 'groupe_d_age'})
            groupe_age_col = 'groupe_d_age'
        df_long.insert(0, 'indicateur', 'Population âgée de 10 ans et plus selon le niveau d’instruction')

        df_long['sexe'] = sexe
        df_long['milieu'] = milieu
        df_long['annee'] = annee
        df_long['unite'] = unite
        df_long['feuille'] = feuille

        dfs_long.append(df_long)
        print(f"✅ Feuille '{feuille}' traitée.\n")

    except Exception as e:
        print(f"❌ Erreur dans la feuille '{feuille}': {e}\n")

if dfs_long:
    df_final = pd.concat(dfs_long, ignore_index=True)
    print("✅ Fusion terminée. Exemple :\n")
    print(df_final.head())
    df_final.to_excel("resultat_educ_auto.xlsx", index=False)
    print("📁 Données enregistrées dans 'resultat_educ_auto.xlsx'")

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df_final)
else:
    print("❌ Aucune donnée traitée.")
df_final.to_csv("instruction_1994.csv", index=False, encoding='utf-8-sig')


📄 Traitement de la feuille : T72-55
✅ Feuille 'T72-55' traitée.

📄 Traitement de la feuille : T72-56
✅ Feuille 'T72-56' traitée.

📄 Traitement de la feuille : T72-57
✅ Feuille 'T72-57' traitée.

📄 Traitement de la feuille : T72-58
✅ Feuille 'T72-58' traitée.

📄 Traitement de la feuille : T72-59
✅ Feuille 'T72-59' traitée.

📄 Traitement de la feuille : T72-60
✅ Feuille 'T72-60' traitée.

📄 Traitement de la feuille : T72-61
✅ Feuille 'T72-61' traitée.

📄 Traitement de la feuille : T72-62
✅ Feuille 'T72-62' traitée.

📄 Traitement de la feuille : T72-63
✅ Feuille 'T72-63' traitée.

📄 Traitement de la feuille : T72-64
✅ Feuille 'T72-64' traitée.

📄 Traitement de la feuille : T72-65
✅ Feuille 'T72-65' traitée.

📄 Traitement de la feuille : T72-66
✅ Feuille 'T72-66' traitée.

📄 Traitement de la feuille : T72-67
✅ Feuille 'T72-67' traitée.

📄 Traitement de la feuille : T72-68
✅ Feuille 'T72-68' traitée.

📄 Traitement de la feuille : T72-69
✅ Feuille 'T72-69' traitée.

📄 Traitement de la feuill